ANOVA

In [1]:
import numpy as np

def means(X):
    res = [];
    for i in range(len(X)):
        res.append(np.mean(X[i]))
    return res

def ANOVA1_partition_TSS(X, silent = True):
    within_means= means(X)
    x_bar = np.mean(within_means)
    sst = 0
    ssw = 0
    ssb = 0
    I = len(X);
    for i in range(I):
        n_i = len(X[i])
        for j in range(n_i):
            sst = sst + (X[i][j] - x_bar)**2
            ssw = ssw + (X[i][j] - within_means[i])**2
        ssb = ssb + n_i * (within_means[i] - x_bar)**2
    if not silent:
        print(sst)
        print(ssw + ssb)
        print(ssw)
        print(ssb)
    return [sst, ssw, ssb]
    
X = [[1,1,1,1,2], [1,1,1], [1,1,1,5,5], [1,1,1,1,2,1,1,1,1]];
n=0
n_is = []
for i in range(len(X)):
    n_is.append(len(X[i]))
    n += len(X[i])
a,b,c = ANOVA1_partition_TSS(X, False);


29.466419753086416
29.466419753086413
20.888888888888882
8.577530864197533


In [2]:
from scipy import stats
from tabulate import tabulate

def ANOVA1_test_equality(X, alpha, n):
    sst, ssw, ssb = ANOVA1_partition_TSS(X)
    I = len(X)
    p_value = 1-stats.f.cdf((ssb/(I-1))/(ssw/(n-I)), I-1, n-I)
    print('P-value is {}'.format(p_value))
    f_statistic = (ssb/(I-1))/(ssw/(n-I))
    crit_value = stats.f.ppf(1-alpha, I-1, n-I)

    print('f-statistic is {}'.format(f_statistic))
    print('Critical value is {}'.format(crit_value)) 

    if p_value < alpha:
        print("Null hypothesis is rejected")
    else:
        print("Null hypothesis has not been rejected\n")

    print(tabulate([
                    ['Between groups', I-1, ssb, ssb/(I-1), (ssb/(I-1))/(ssw/(n-I))],
                    ['Within groups', n-I, ssw, ssw/(n-I)],
                    ['Total', n-1, sst]], 
                    headers=['Source', 'DF', 'SS', 'MS', 'F'], tablefmt='orgtbl'))

ANOVA1_test_equality(X, 0.05, n)

P-value is 0.09552454602798377
f-statistic is 2.4637588652482285
Critical value is 3.1599075898007243
Null hypothesis has not been rejected

| Source         |   DF |       SS |      MS |       F |
|----------------+------+----------+---------+---------|
| Between groups |    3 |  8.57753 | 2.85918 | 2.46376 |
| Within groups  |   18 | 20.8889  | 1.16049 |         |
| Total          |   21 | 29.4664  |         |         |


In [3]:
def ANOVA1_is_contrast(c, I, silent=True):
    if len(c) != I:
        print("Dimension mismatch")
    sum = np.sum(c)
    if sum == 0:
        if not silent:
            print("it is a contrast")
        return True
    else:
        if not silent:
            print("it is not a contrast")
        return False

c_example = [1,-1,0,0]
ANOVA1_is_contrast(c_example, len(X))

True

In [6]:
def ANOVA1_is_orthogonal(c1, c2, n_is, silent=True):
    if not (ANOVA1_is_contrast(c1, len(n_is)) and ANOVA1_is_contrast(c2,len(n_is))):
        print("Warning: At least one of the given vectors is not a contrast")
    I = len(c1)
    inner_prod = 0
    for i in range(I):
        inner_prod += c1[i]*c2[i]/n_is[i]
    
    if inner_prod == 0:
        if not silent:
            print("They are orthogonal")
        return True
    else:
        if not silent:
            print("They are not orthogonal")
        return False

c1 = [1,-1,0,0]
c2 = [0,0,1,-1]

ANOVA1_is_orthogonal(c1,c2,n_is)


True

In [7]:
def Bonferroni_correction(fwer_alpha, m, silent=True):
    if not silent:
        print("The significance level that each test must have with Bonferroni correction is", fwer_alpha/m)
    return fwer_alpha/m

Bonferroni_correction(0.05, 4)


0.0125

In [8]:
def Sidak_correction(fwer_alpha, m, silent=True):
    if not silent:
        print("The significance level that each test must have with Sidak correction is", 1-(1-fwer_alpha)**(1/m))
    return 1-(1-fwer_alpha)**(1/m)

Sidak_correction(0.05, 4)


0.012741455098566168

In [9]:
import math
from statsmodels.stats.libqsturng import psturng, qsturng

def is_pairwise_diff(C, I):
    if len(set(n_is)) != 1:
        return False
    if len(C) > I*(I-1)/2:
        return False
    else:
        for linear_com in C:
            if sum(linear_com) != 0:
                return False
            count_neg_ones = 0
            count_ones = 0
            for el in linear_com:
                if el == -1:
                    count_neg_ones += 1
                elif el == 1:
                    count_ones += 1
                elif el != 0:
                    return False
            if not (count_neg_ones == 1 and count_ones == 1):
                return False
        return True 

# example qsturng(1-0.05, 10, 20)

def ANOVA1_CI_linear_combs(X, n_is, n, alpha, C, method, silent=True):
    I = len(X)
    all_contrasts = True
    for i in range(len(C)):
        all_contrasts = (all_contrasts and ANOVA1_is_contrast(C[i], I))

    all_orthogonal = True
    for i in range(len(C)): # look all pairwise orthogonality
        for j in range(len(C)):
            if i != j:
                all_orthogonal = (all_orthogonal and ANOVA1_is_orthogonal(C[i], C[j], n_is))

    all_pairwise_diff = is_pairwise_diff(C, I)
    #if not silent:
    #    print("All orthogonal", all_orthogonal)
    #    print("All contrasts", all_contrasts)
    sst, ssw, ssb = ANOVA1_partition_TSS(X)
    x_bar_is = means(X)

    squared_terms = []
    for m in range(len(C)):
        squared_term = 0
        for i in range(I):
            squared_term += C[m][i]**2/len(X[i]) 
        squared_term *= ssw/(n-I)
        squared_terms.append(squared_term**(1/2))
    
    if method == "scheffe":
        CIs= []
        if all_contrasts:
            # theorem 2.8 (scheffe's theorem for all contrasts)
            M_val = ((I-1) * stats.f.ppf(1-alpha, I-1, n-I))**(1/2)
            for j in range(len(C)): # m-ary CI
                sum = 0
                for i in range(I):
                    sum += C[j][i]*x_bar_is[i]
                #print(sum)
                #print(M_val)
                #print(squared_terms[j])
                CI = [sum - M_val * squared_terms[j], sum + M_val * squared_terms[j]]
                CIs.append(CI)

        else:
            # theorem 2.7 (scheffe's theorem for all linear combinations)
            M_val = ((I-1) * stats.f.ppf(1-alpha, I, n-I))**(1/2)
            for j in range(len(C)): # m-ary CI
                sum = 0
                for i in range(I):
                    sum += C[j][i]*x_bar_is[i]
                CI = [sum - M_val * squared_terms[j], sum + M_val * squared_terms[j]]
                CIs.append(CI)
        if not silent:
            print("Confidence interval with {0} is {1}".format(method, CIs))
        return CIs
        

    elif method == "tukey":
        # check if all linear combs are pairwise differences (and also check n_i s are equal)
        if all_pairwise_diff:
            pair_differences = np.dot(C, np.transpose(x_bar_is))
            CIs = []
            # std_rng_statistic = np.max(pair_differences)
            sqrd_term = (ssw/((n-I)*len(X[0])))**(1/2)
            q = qsturng(1-alpha, I, n-I)
            print(sqrd_term)
            print(q)
            for pair_diff in pair_differences:
                CI = [pair_diff - q*sqrd_term, pair_diff + q*sqrd_term]
                CIs.append(CI)
            if not silent:
                print("Confidence interval with {0} is {1}".format(method, CIs))
            return CIs
        else:
            print("Given linear combinations do not consist of pairwise differences")
            
    elif method == "bonferroni":
        CIs = []
        b_alpha = Bonferroni_correction(alpha, len(C))
        t_val = stats.t.ppf(1-b_alpha, n-I)
        
        for j in range(len(C)): # m-ary CI
            sum = 0
            for i in range(I):
                sum += C[j][i]*x_bar_is[i]
            CI = [sum - t_val * squared_terms[j], sum + t_val * squared_terms[j]]
            CIs.append(CI)
        if not silent:
            print("Confidence interval with {0} is {1}".format(method, CIs))
        return CIs
        
    elif method == "sidak":
        if all_contrasts and all_orthogonal:
            CIs = []
            s_alpha = Sidak_correction(alpha, len(C))
            t_val = stats.t.ppf(1-s_alpha, n-I)
            for j in range(len(C)): # m-ary CI
                sum = 0
                for i in range(I):
                    sum += C[j][i]*x_bar_is[i]
                CI = [sum - t_val * squared_terms[j], sum + t_val * squared_terms[j]]
                CIs.append(CI)
            if not silent:
                print("Simultaneous Confidence intervals with {0} is {1}".format(method, CIs))
            return CIs
        else:
            print("We cannot use Sidak's method if given linear combinations are not orthogonal contrasts")
    elif method == "best":
        if all_contrasts and all_orthogonal:
            # compare sidak with thm 2.8(scheffe contrast)
            schf_CIs = np.array(ANOVA1_CI_linear_combs(X, n_is, n, alpha, C, "scheffe"))
            sdk_CIs = np.array(ANOVA1_CI_linear_combs(X, n_is, n, alpha, C, "sidak"))
            if np.sum(sdk_CIs[:,0]-sdk_CIs[:,0]) > np.sum(schf_CIs[:,1] - schf_CIs[:,0]):
                if not silent:
                    print("Best confidence interval is with scheffe contrast: {0}".format(schf_CIs))
                return schf_CIs
            else:
                if not silent:
                    print("Best confidence interval is with sidak: {0}".format(sdk_CIs))
                return sdk_CIs

        elif all_contrasts and not all_orthogonal:
            # compare bonferroni with thm 2.8 (scheffe contrast)
            schf_CIs = np.array(ANOVA1_CI_linear_combs(X, n_is, n, alpha, C, "scheffe"))
            bnf_CIs = np.array(ANOVA1_CI_linear_combs(X, n_is, n, alpha, C, "bonferroni"))
            if np.sum(schf_CIs[:,1] - schf_CIs[:,0]) < np.sum(bnf_CIs[:,1] - bnf_CIs[:,0]):
                if not silent:
                    print("Best confidence interval is with scheffe contrast: {0}".format(schf_CIs))
                return schf_CIs
            else:
                if not silent:
                    print("Best confidence interval is with bonferroni: {0}".format(bnf_CIs))
                return bnf_CIs

        elif all_pairwise_diff and all_orthogonal:
            # compare sidak with tukey
            tky_CIs = np.array(ANOVA1_CI_linear_combs(X, n_is, n, alpha, C, "tukey"))
            sdk_CIs = np.array(ANOVA1_CI_linear_combs(X, n_is, n, alpha, C, "sidak"))
            if np.sum(tky_CIs[:,1] - tky_CIs[:,0]) < np.sum(sdk_CIs[:,1] - sdk_CIs[:,0]):
                if not silent:
                    print("Best confidence interval is with tukey: {0}".format(tky_CIs))
                return tky_CIs
            else:
                if not silent:
                    print("Best confidence interval is with sidak: [{0}, {1}]".format(sdk_CIs))
                return sdk_CIs

        elif all_pairwise_diff and not all_orthogonal:
            # compare bonferroni with tukey
            tky_CIs = np.array(ANOVA1_CI_linear_combs(X, n_is, n, alpha, C, "tukey"))
            bnf_CIs = np.array(ANOVA1_CI_linear_combs(X, n_is, n, alpha, C, "bonferroni"))
            if np.sum(tky_CIs[:,1] - tky_CIs[:,0]) < np.sum(bnf_CIs[:,1] - bnf_CIs[:,0]):
                if not silent:
                    print("Best confidence interval is with tukey: {0}".format(tky_CIs))
                return tky_CIs
            else:
                if not silent:
                    print("Best confidence interval is with bonferroni: [{0}, {1}]".format(bnf_CIs))
                return bnf_CIs
        else:
            # compare bonferroni with thm 2.7 (scheffe linaer combs)
            schf_CIs = np.array(ANOVA1_CI_linear_combs(X, n_is, n, alpha, C, "scheffe"))
            bnf_CIs = np.array(ANOVA1_CI_linear_combs(X, n_is, n, alpha, C, "bonferroni"))
            if np.sum(schf_CIs[:,1] - schf_CIs[:,0]) < np.sum(sdk_CIs[:,1] - sdk_CIs[:,0]):
                if not silent:
                    print("Best confidence interval is with scheffe linear combs: {0}".format(schf_CIs))
                return schf_CIs
            else:
                if not silent:
                    print("Best confidence interval is with bonferroni: {0}".format(bnf_CIs))
                return bnf_CIs
    else:
        print("Not implemented error")
        return -1e6, 1e6

C= [c1, c2]
print("Scheffe:",ANOVA1_CI_linear_combs(X, n_is, n, 0.1, C, "scheffe", True))
print("Bonferroni:",ANOVA1_CI_linear_combs(X, n_is, n, 0.1, C, "bonferroni", True))
print("Sidak:",ANOVA1_CI_linear_combs(X, n_is, n, 0.1, C, "sidak", True))
print("Tukey:",ANOVA1_CI_linear_combs(X, n_is, n, 0.1, C, "tukey", True))
print("Best:",ANOVA1_CI_linear_combs(X, n_is, n, 0.1, C, "best", True))

Scheffe: [[-1.9180214641411217, 2.318021464141122], [-0.1287767249390923, 3.10655450271687]]
Bonferroni: [[-1.1642243751318286, 1.5642243751318285], [0.44694531156464956, 2.5308324662131283]]
Sidak: [[-1.1529289484907017, 1.5529289484907016], [0.4555723361629471, 2.5222054416148305]]
Given linear combinations do not consist of pairwise differences
Tukey: None
Best: [[-1.15292895  1.55292895]
 [ 0.45557234  2.52220544]]


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [10]:
def ANOVA1_test_linear_combs(X, n_is, n, alpha, C, d, method, silent=True):
    CIs = ANOVA1_CI_linear_combs(X, n_is, n, alpha, C, method)
    for i in range(len(C)):
        el = d[i]
        #print(CIs, i)
        if CIs[i][0] < el and el < CIs[i][1]:
            print("We do not reject H0 for linear comb {}".format(i+1))
        else:
            print("We reject H0 for linear comb {}".format(i+1))

d = [2]*len(C)
ANOVA1_test_linear_combs(X, n_is, n, 0.1, C, d, "best", False)


We reject H0 for linear comb 1
We do not reject H0 for linear comb 2


In [12]:
def ANOVA2_partition_TSS(X):
    I, J, K = np.shape(X)
    x_bar_ij = np.zeros(shape=(I, J))
    x_bar_i = np.zeros(shape=(I))
    x_bar_j = np.zeros(shape=(J))
    x_bar = 0
    for i in range(I):
        res_i = 0
        for j in range(J):
            res_ij = 0
            for k in range(K):
                res_ij += X[i][j][k]
                res_i += X[i][j][k]
                x_bar += X[i][j][k]
            res_ij /= K
            x_bar_ij[i][j] = res_ij
        res_i /= (J*K)
        x_bar_i[i] = res_i
    x_bar /= (I*J*K)
    for j in range(J):
        res_j = 0
        for i in range(I):
            for k in range(K):
                res_j += X[i][j][k]
        res_j /= (I*K)
        x_bar_j[j] = res_j
        
    sst = ssa = ssb = ssab = sse = 0
    for i in range(I):
        for j in range(J):
            ssab += (x_bar_ij[i][j] - x_bar_i[i] - x_bar_j[j] + x_bar)**2
            for k in range(K):
                sst += (X[i][j][k] - x_bar)**2
                sse += (X[i][j][k] - x_bar_ij[i][j])**2

    ssab *= K
    for i in range(I):
        ssa += (x_bar_i[i] - x_bar)**(2)
    ssa *= (J*K)
    for j in range(J):
        ssb += (x_bar_j[j] - x_bar)**(2)
    ssb *= (I*K)
    return sst, ssa, ssb, ssab, sse, x_bar, x_bar_i, x_bar_j, x_bar_ij


# testing
# X = np.random.randint(0, 100, size=(30, 10, 5))
X = [[[8,7,7,6],[5, 6, 6, 4]],[[13,15,14,15],[12, 14, 13, 14]]]
sst, ssa, ssb, ssab, sse, _, _, _, _ = ANOVA2_partition_TSS(X)
print(sst, ssa, ssb, ssab, sse)
print(sst, ssa+ssb+ssab+sse)

250.9375 232.5625 7.5625 0.5625 10.25
250.9375 250.9375


In [13]:
def ANOVA2_MLE(X):
    I, J, K = np.shape(X)
    _, _, _, _, _, x_bar, x_bar_i, x_bar_j, x_bar_ij = ANOVA2_partition_TSS(X)
    mu = x_bar
    ais = x_bar_i - x_bar
    bjs = x_bar_j - x_bar
    dijs = np.zeros(shape=(I, J))
    for i in range(I):
        for j in range(J):
            dijs[i][j] = x_bar_ij[i][j] - x_bar_i[i] - x_bar_j[j] + x_bar
    return x_bar, ais, bjs, dijs

xb, ai, bi, dij = ANOVA2_MLE(X)
print(xb)
print(ai)
print(bi)
print(dij)


9.9375
[-3.8125  3.8125]
[ 0.6875 -0.6875]
[[ 0.1875 -0.1875]
 [-0.1875  0.1875]]


In [14]:
def ANOVA2_test_equality(X, alpha, choice):
    sst, ssa, ssb, ssab, sse, _, _, _, _ = ANOVA2_partition_TSS(X)
    I,J,K = np.shape(X)
    dof_within = I*J*(K-1)
    if choice == "A":
        dof_a = I-1
        f_statistic = (ssa/dof_a)/(sse/dof_within)
        p_value = 1-stats.f.cdf(f_statistic, dof_a, dof_within)
        crit_value = stats.f.ppf(1-alpha, dof_a, dof_within)

        print('P-value is {}'.format(p_value))
        print('f-statistic is {}'.format(f_statistic))
        print('Critical value is {}'.format(crit_value)) 
        if p_value < alpha:
            print("Null hypothesis is rejected")
        else:
            print("Null hypothesis has not been rejected\n")

        print(tabulate([
                    ['A', dof_a, ssa, ssa/dof_a, f_statistic],
                    ['Within', dof_within, sse, sse/dof_within]],
                    headers=['Source', 'degrees of freedom', 'SS', 'MS', 'F'], tablefmt='orgtbl'))
        
    elif choice == "B":
        dof_b = J-1
        f_statistic = (ssb/dof_b)/(sse/dof_within)
        p_value = 1-stats.f.cdf(f_statistic, dof_b, dof_within)
        crit_value = stats.f.ppf(1-alpha, dof_b, dof_within)

        print('P-value is {}'.format(p_value))
        print('f-statistic is {}'.format(f_statistic))
        print('Critical value is {}'.format(crit_value)) 
        if p_value < alpha:
            print("Null hypothesis is rejected")
        else:
            print("Null hypothesis has not been rejected\n")

        print(tabulate([
                    ['B', dof_b, ssb, ssb/dof_b, f_statistic],
                    ['Within', dof_within, sse, sse/dof_within]],
                    headers=['Source', 'degrees of freedom', 'SS', 'MS', 'F'], tablefmt='orgtbl'))

    elif choice == "AB":
        dof_ab = (I-1)*(J-1)
        f_statistic = (ssab/dof_ab)/(sse/dof_within)
        p_value = 1-stats.f.cdf(f_statistic, dof_ab, dof_within)
        crit_value = stats.f.ppf(1-alpha, dof_ab, dof_within)

        print('P-value is {}'.format(p_value))
        print('f-statistic is {}'.format(f_statistic))
        print('Critical value is {}'.format(crit_value)) 
        if p_value < alpha:
            print("Null hypothesis is rejected")
        else:
            print("Null hypothesis has not been rejected\n")

        print(tabulate([
                    ['AxB', dof_ab, ssab, ssab/dof_ab, f_statistic],
                    ['Within', dof_within, sse, sse/dof_within]],
                    headers=['Source', 'degrees of freedom', 'SS', 'MS', 'F'], tablefmt='orgtbl'))
    print("\n/////////////////////////////////////////////////////////////////////////\n")

print("/////////////////////////////////////////////////////////////////////////")
ANOVA2_test_equality(X, 0.05, "A")
ANOVA2_test_equality(X, 0.05, "B")
ANOVA2_test_equality(X, 0.05, "AB")

/////////////////////////////////////////////////////////////////////////
P-value is 1.3002787735416632e-09
f-statistic is 272.26829268292687
Critical value is 4.747225346722511
Null hypothesis is rejected
| Source   |   degrees of freedom |      SS |         MS |       F |
|----------+----------------------+---------+------------+---------|
| A        |                    1 | 232.562 | 232.562    | 272.268 |
| Within   |                   12 |  10.25  |   0.854167 |         |

/////////////////////////////////////////////////////////////////////////

P-value is 0.0115819925482612
f-statistic is 8.853658536585366
Critical value is 4.747225346722511
Null hypothesis is rejected
| Source   |   degrees of freedom |      SS |       MS |       F |
|----------+----------------------+---------+----------+---------|
| B        |                    1 |  7.5625 | 7.5625   | 8.85366 |
| Within   |                   12 | 10.25   | 0.854167 |         |

//////////////////////////////////////////////

LINEAR REGRESSION


In [15]:
def Mult_LR_Least_squares(X, y):
    # produces the maximum likelihood estimators for β and σ 2 as well as the unbiased estimate for σ
    n = len(X)
    k_plus_one = len(X[0])
    beta = np.dot(np.dot(np.linalg.inv(np.dot(np.transpose(X),X)), np.transpose(X)), y)
    squared_error = np.dot(np.transpose(y- np.dot(X, beta)), (y - np.dot(X,beta)))
    s_sq_mle = squared_error / n
    s_sq_unb = squared_error / (n - k_plus_one)
    return beta, s_sq_mle, s_sq_unb

# took the example from hw3
X = [[1, 42.8, 40.0], [1, 63.5, 93.5], [1, 37.5, 35.5], [1, 39.5, 30.0], [1, 45.5, 52.0], [1, 38.5, 17.0], [1, 43.0, 38.5], [1, 22.5, 8.5], [1, 37.0, 33.0], [1, 23.5, 9.5], [1, 33.0, 21.0], [1, 58.0, 79.0]]
y = [37.0, 49.5, 34.5, 36.0, 43.0, 28.0, 37.0, 20.0, 33.5, 30.5, 38.5, 47.0]
X = np.array(X)
y = np.array(y)
print(Mult_LR_Least_squares(X,y))

(array([21.00839777,  0.19635663,  0.19082778]), 11.659419728755699, 15.545892971674265)


In [16]:
def Mult_LR_partition_TSS(X, y):
    # returns the total sum of squares, regression sum of squares, and residual sum of squares
    n = len(X)
    k_plus_one = len(X[0])
    beta, _, _ = Mult_LR_Least_squares(X,y)
    ybar = np.mean(y)
    tss = reg_ss = rss = 0
    for i in range(n):
        truth = y[i]
        est = np.dot(X[i], beta)
        rss += (truth - est)**2
        reg_ss += (est - ybar)**2
        tss += (truth - ybar)**2
    return tss, reg_ss, rss

tss, reg_ss, rss = Mult_LR_partition_TSS(X,y)
print(tss, reg_ss, rss, reg_ss+rss)


718.7291666666665 578.8161299215975 139.9130367450684 718.729166666666


In [17]:
def Mult_norm_LR_simul_CI(X, y, alpha):
    # produces confidence intervals for βi’s that simultaneously hold with probability 1 − α
    n = len(X)
    k_plus_one = len(X[0])
    beta, _, se_sqrd = Mult_LR_Least_squares(X,y)
    M = se_sqrd**(1/2) * (k_plus_one * stats.f.ppf(1-alpha, k_plus_one, n-k_plus_one))**(1/2)
    inv_xt_x = np.linalg.inv(np.dot(np.transpose(X),X))
    CI_for_all = []
    # print(beta)
    for i in range(k_plus_one):
        margin = M * inv_xt_x[i][i]**(1/2)
        CI = [beta[i] - margin, beta[i] + margin]
        CI_for_all.append(CI)

    return CI_for_all

CI_for_all = Mult_norm_LR_simul_CI(X, y, 0.1)
print(CI_for_all)


[[-4.413054294605068, 46.42984984366482], [-0.8511143664765652, 1.2438276311981367], [-0.28896120320521657, 0.6706167645312096]]


In [18]:
def Mult_norm_LR_CR(X, y, C, alpha):
    # returns the specifications (that is, parameters of the ellipsoid) of the 100(1 − α)% 
    # confidence region for Cβ according to the normal multiple linear regression model.
    n = len(X)
    k_plus_one = len(X[0])
    beta, _, se_sqrd = Mult_LR_Least_squares(X,y)
    r = len(C)
    thold = r * se_sqrd * stats.f.ppf(1-alpha, r, n-k_plus_one)
    inv_xt_x = np.linalg.inv(np.dot(np.transpose(X),X))
    # print(np.dot(C, inv_xt_x))
    inv_c_invxtx_c = np.linalg.inv(np.dot(np.dot(C, inv_xt_x), np.transpose(C)))
    c_beta = np.dot(C, beta)
    return c_beta, inv_c_invxtx_c, thold 

C = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
# C = [[1, 0, 0]]

Mult_norm_LR_CR(X, y, C, 0.1)

(array([21.00839777,  0.19635663,  0.19082778]),
 array([[1.200000e+01, 4.843000e+02, 4.575000e+02],
        [4.843000e+02, 2.111259e+04, 2.175200e+04],
        [4.575000e+02, 2.175200e+04, 2.491125e+04]]),
 131.18540129454095)

In [19]:
def Mult_norm_LR_is_in_CR(X, y, C, c0, alpha, silent=False):
    # answers whether c0 is in the 100(1 − α)% confidence region for Cβ according to the
    # normal multiple linear regression model
    c_beta, inv_c_invxtx_c, thold = Mult_norm_LR_CR(X, y, C, alpha)
    print(c_beta - c0, inv_c_invxtx_c, thold)
    if np.dot(np.dot(np.transpose(c_beta-c0), inv_c_invxtx_c), c_beta-c0) > thold:
        if not silent:
            print("c0 is not inside the confidence region")
        return False
    else:
        if not silent:
            print("c0 is inside the confidence region")
        return True

c0 = [21, 0.20, 0.20]
Mult_norm_LR_is_in_CR(X,y,C,c0,0.1)

[ 0.00839777 -0.00364337 -0.00917222] [[1.200000e+01 4.843000e+02 4.575000e+02]
 [4.843000e+02 2.111259e+04 2.175200e+04]
 [4.575000e+02 2.175200e+04 2.491125e+04]] 131.18540129454095
c0 is inside the confidence region


True

In [20]:
def Mult_norm_LR_test_general(X, y, C, c0, alpha):
    # tests the null hypothesis H0 : Cβ = c0 vs H1 : Cβ 6= c0 at a significance level of α
    res = Mult_norm_LR_is_in_CR(X, y, C, c0, alpha, True)
    if res:
        print("Do not reject H0")
    else:
        print("Reject H0")
Mult_norm_LR_test_general(X,y,C,c0,0.1)

[ 0.00839777 -0.00364337 -0.00917222] [[1.200000e+01 4.843000e+02 4.575000e+02]
 [4.843000e+02 2.111259e+04 2.175200e+04]
 [4.575000e+02 2.175200e+04 2.491125e+04]] 131.18540129454095
Do not reject H0


In [21]:
def Mult_norm_LR_test_comp(X, y, alpha, js, silent=False):
    # returns the outcome of testing H0 : βj1 = . . . = βjr = 0 vs H1 : not H0
    C = np.zeros(shape =(len(js),len(X[0])))
    for i in range(len(js)):
        C[i][js[i]]=1

    c0 = [0] * len(js)

    Mult_norm_LR_test_general(X,y,C,c0,alpha)
    '''
    idx = []
    for i in range(len(X[0])):
        if i not in js:
            idx.append(i)
    u_j = X[:,idx]

    nof_zeros = len(js) - np.sum(js)
    n = len(X)
    k_plus_one = len(X[0])
    tss, reg_ss, rss = Mult_LR_partition_TSS(X,y)
    beta_j = np.dot(np.dot(np.linalg.inv(np.dot(np.transpose(u_j),u_j)), np.transpose(u_j)), y)
    ybar = np.mean(y)
    reg_ss_j = 0
    for i in range(n):
        est_j = np.dot(u_j[i], beta_j)
        reg_ss_j += (est_j - ybar)**2
    
    reg_ss_residual = reg_ss - reg_ss_j
    dof_reg_ss = len(js)
    dof_rss = n - k_plus_one 
    f_statistic = (reg_ss_residual/dof_reg_ss) / (rss/dof_rss)
    crit_val = stats.f.ppf(1-alpha, dof_reg_ss, dof_rss)

    if f_statistic > crit_val:
        if not silent:
            print("We reject H0, (Bj1... Bjr are significant)")
        return False
    else:
        if not silent:
                print("We do not reject H0, (Bj1... Bjr are insignificant)")
        return True

'''

js = [2]
Mult_norm_LR_test_comp(X, y, 0.1, js)

[0.19082778] [[569.88203146]] 52.23891116150051
Do not reject H0


In [22]:
def Mult_norm_LR_test_linear_reg(X, y, alpha, silent=False):
    # returns the outcome of testing the existence of linear regression
    # at all, i.e., H0 : β1 = . . . = βk = 0 vs H1 : not H0
    tss, reg_ss, rss = Mult_LR_partition_TSS(X, y)
    n = len(X)
    k_plus_one = len(X[0])
    beta, _, se_sqrd = Mult_LR_Least_squares(X,y)

    f_statistic = (reg_ss / (k_plus_one - 1)) / (rss/(n - k_plus_one))
    t_hold = stats.f.ppf(1-alpha, k_plus_one - 1, n-k_plus_one)
    if f_statistic > t_hold:
        if not silent:
            print("We reject H0, (there is a linear regression, beta is significant)")
        return False
    else:
        if not silent:
            print("We do not reject H0, (there is not a linear regression at all, beta is insignificant)")
        return True

Mult_norm_LR_test_linear_reg(X, y, 0.1)

We reject H0, (there is a linear regression, beta is significant)


False

In [23]:
def Mult_norm_LR_pred_CI(X, y, D, alpha, method):
    # returns simultaneous confidence bounds for diβ for all i = 1, . . . , m 
    # according to the normal multiple linear regression model
    n = len(X)
    k_plus_one = len(X[0])
    beta, _, se_sqrd = Mult_LR_Least_squares(X, y)
    M = se_sqrd**(1/2) * (k_plus_one * stats.f.ppf(1-alpha, k_plus_one, n-k_plus_one))**(1/2)
    inv_xt_x = np.linalg.inv(np.dot(np.transpose(X),X))


    if method == "scheffe":
        CIs= []
        for d in D:
            est = np.dot(d, beta)
            margin = np.dot(np.dot(np.transpose(d), inv_xt_x), d)**(1/2) * M
            CIs.append([est-margin, est+margin])
        return CIs

    elif method == "bonferroni":
        CIs = []
        CIs_for_beta = []
        bonf_alpha = Bonferroni_correction(alpha, len(D)) #alpha / len(D)
        for i in range(k_plus_one):
            margin = stats.t.ppf(1-bonf_alpha/2, n-k_plus_one) * se_sqrd**(1/2) * inv_xt_x[i][i]**(1/2)
            CI = [beta[i] - margin, beta[i] + margin]
            CIs_for_beta.append(CI)
        CIs_for_beta = np.array(CIs_for_beta)
        # print(CIs_for_beta)
        # CIs_for_beta = np.array(Mult_norm_LR_simul_CI(X,y,alpha)) # .reshape((3,2))
        # print(CIs_for_beta)
        beta_low = CIs_for_beta[:,0]
        beta_high = CIs_for_beta[:,1]
        for d in D:
            est_low = np.dot(d, beta_low)
            est_high = np.dot(d, beta_high)
            CIs.append([est_low, est_high])
        return CIs

    elif method == "best":
        CIs = []
        CIs_bonf = Mult_norm_LR_pred_CI(X, y, D, alpha, "bonferroni")
        CIs_schf = Mult_norm_LR_pred_CI(X, y, D, alpha, "scheffe")
        for i in range(len(D)):
            bnf = CIs_bonf[i]
            schf = CIs_schf[i]
            if (schf[1] - schf[0]) < (bnf[1] - bnf[0]):
                CIs.append(schf)
            else:
                CIs.append(bnf)
        return CIs

m = 2
# D = np.random.randint(90, size=(m, 3))
D= np.array([[84, 23, 53],
 [18, 67, 87]])
D[:,0] = 1 # to make first column 1
print(D)

print(Mult_norm_LR_pred_CI(X, y, D, 0.1, "bonferroni"))
print(Mult_norm_LR_pred_CI(X, y, D, 0.1, "scheffe"))
print(Mult_norm_LR_pred_CI(X, y, D, 0.1, "best"))



[[ 1 23 53]
 [ 1 67 87]]
[[-22.721317539727057, 93.99826292766058], [-56.18754613228031, 157.72016425304676]]
[[10.302781502063901, 60.97416388586962], [41.726703170347534, 59.80591495041891]]
[[10.302781502063901, 60.97416388586962], [41.726703170347534, 59.80591495041891]]
